In [ ]:
# set parameter for training
task = "PATH_MODEL_0"
id_ = "PATH_MODEL_1"
dir_path = f'../save_model/{task}/{id_}'

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from drs import util, dataset

import torch 
import argparse
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])

# embedding path
args.embeddimg_path = f'../data/Fasttext_Multi_300.mdl'
# data path
args.path_atc = f"../data/list_med.pkl"

args.valid_path = f"../data/data_valid.pkl"
args.test_path = f"../data/data_test.pkl"

# adj DDI path
args.major_ddi_path = f"../data/adj_ddi_major.npy"
args.moder_ddi_path = f"../data/adj_ddi_moder.npy"
args.adj_ddi_major = torch.tensor(np.load(args.major_ddi_path), dtype=torch.float32).cuda()
args.adj_ddi_moder = torch.tensor(np.load(args.moder_ddi_path), dtype=torch.float32).cuda()

# parameters
args.note = True
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.seed = 42
args.seq_length = 1024
args.batch_size = 128
util.set_seed(args.seed)

In [ ]:
args.embedding_matrix, args.word2idx = util.word2matrix(args)
args.mapping, args.num_classes_layer, args.total_classes = util.class_label(args.path_atc)
args.int_to_vocab = {v: k for k, v in args.word2idx.items()}

In [ ]:
model_path = f'../save_model/{task}/{id_}/model_best_{id_}.pth'
model = torch.load(model_path, weights_only=False).to('cuda')

In [ ]:
# define valid dataset
valid_dataset = dataset.define_ds(args, data_path = args.valid_path)
valid_dataloader  = dataset.define_dl(valid_dataset, batch_size=args.batch_size)

In [ ]:
val_true = torch.tensor([], device=args.device)
val_pred = torch.tensor([], device=args.device)

model.eval()
with torch.no_grad():
    for X_val, y_val in tqdm(valid_dataloader):
        _, scores = model(X_val.to('cuda'))
        val_true = torch.cat((val_true, y_val.to('cuda')), dim=0)
        val_pred = torch.cat((val_pred, scores), dim=0)
val_true = val_true.cpu().detach().numpy()
val_pred = val_pred.cpu().detach().numpy()

# save validation true and predict
np.save(f"{dir_path}/val_true.npy", val_true)
np.save(f"{dir_path}/val_pred.npy", val_pred)

In [ ]:
# per-label threshold
best_thr_each_class = []
for (i_val_true, i_val_pred) in tqdm(zip(np.transpose(val_true),np.transpose(val_pred)), total=val_true.shape[1]):
    best_threshold, best_f1 = util.find_best_threshold(i_val_true, i_val_pred)
    best_thr_each_class.append(best_threshold)
# global threshold
global_best_threshold, global_best_f1 = util.find_best_threshold(val_true, val_pred)

# save Threshold
np.save(f"{dir_path}/best_thr_each_class.npy", best_thr_each_class) 
np.save(f"{dir_path}/global_best_threshold.npy", global_best_threshold) 

In [ ]:
test_dataset = dataset.define_ds(args, data_path = args.path_test)
test_dataloader  = dataset.define_dl(test_dataset, batch_size=args.batch_size)

In [ ]:
y_true = torch.tensor([], device=args.device)
y_pred = torch.tensor([], device=args.device)

model.eval()
with torch.no_grad():
    for X_test, y_test in tqdm(test_dataloader):
        _, scores = model(X_test.to('cuda'))
        y_true = torch.cat((y_true, y_test.to('cuda')), dim=0)
        y_pred = torch.cat((y_pred, scores), dim=0)       
y_true = y_true.cpu().detach().numpy()
y_pred = y_pred.cpu().detach().numpy()

# save  test true and predict
np.save(f"{dir_path}/test_true.npy", y_true)
np.save(f"{dir_path}/test_pred.npy", y_pred)